In [1]:
from qmratool.models import *
#import pymc3 as pm
import plotly.express as px
#import arviz as az
import numpy as np
import pandas as  pd
import plotly.express as px
from plotly.offline import plot
from django_pandas.io import read_frame
import plotly.graph_objects as go
import decimal

In [2]:
def annual_risk(nexposure, event_probs):
    return 1-np.prod(1-np.random.choice(event_probs, nexposure, True))


In [7]:
def simulate_risk(ra):
    # Selecting inflow concentration based in source water type
    df_inflow = read_frame(Inflow.objects.filter(water_source=ra.source).values("min", "max", "pathogen__pathogen", "water_source__water_source_name"))
    df_inflow = df_inflow[df_inflow["pathogen__pathogen"].isin(["Rotavirus", "Campylobacter jejuni", "Cryptosporidium parvum"])]
    # Querying dose response parameters based on pathogen inflow
    dr_models = read_frame(DoseResponse.objects.filter(pathogen__in=Pathogen.objects.filter(pathogen__in=df_inflow["pathogen__pathogen"])))

    # Querying for Logremoval based on selected treatments
    df_treatment=read_frame(LogRemoval.objects.filter(treatment__in=ra.treatment.all()).values("min", "max", "treatment__name", "pathogen_group__pathogen_group"))
    
    # Summarizing treatment to treatment max and treatment min
    df_treatment_summary=df_treatment.groupby(["pathogen_group__pathogen_group"]).sum().reset_index()

    results = pd.DataFrame()

    for index, row in df_inflow.iterrows():
        d = df_inflow.loc[df_inflow["pathogen__pathogen"] ==row["pathogen__pathogen"]]
        dr = dr_models.loc[dr_models["pathogen"]==row["pathogen__pathogen"]]

        if row["pathogen__pathogen"] == "Rotavirus":
            selector = "Viruses" 
        elif row["pathogen__pathogen"]== "Cryptosporidium parvum":
            selector = "Protozoa"
        else:
            selector = "Bacteria"
        #result.append(selector)

        df_treat = df_treatment_summary[df_treatment_summary["pathogen_group__pathogen_group"]==selector]



        risk_df = pd.DataFrame({"inflow": np.random.normal(loc=(np.log10(float(d["min"])+10**(-8))+np.log10(float(d["max"]) ))/2, 
                                                            scale = (np.log10(float(d["max"]))-np.log10(float(d["min"])+10**(-8) ))/4,  
                                                            size = 10000),
                                "LRV": np.random.uniform(low= df_treat["min"], 
                                                         high= df_treat["max"], 
                                                         size= 10000) })
        risk_df["outflow"]=risk_df["inflow"] - risk_df["LRV"]
        risk_df["dose"] = (10**risk_df["outflow"])*float(ra.exposure.volume_per_event)

        if selector != "Protozoa":
            risk_df["probs"] = 1 - (1 + (risk_df["dose"]) * (2 ** (1/float(dr.alpha)) - 1)/float(dr.n50)) ** -float(dr.alpha)
        else:
            risk_df["probs"] = 1 - np.exp(-float(dr.k)*(risk_df["dose"]))

        results[row["pathogen__pathogen"]] = [annual_risk(int(ra.exposure.events_per_year), risk_df["probs"] ) for _ in range(1000)]

    results_long = pd.melt(results)
    results_long["log probability"] = np.log10(results_long["value"])
    return results_long

In [110]:
def simulate_risk(ra):
    # Selecting inflow concentration based in source water type
    df_inflow = read_frame(Inflow.objects.filter(water_source=ra.source).values("min", "max", "pathogen__pathogen", "water_source__water_source_name"))
    df_inflow = df_inflow[df_inflow["pathogen__pathogen"].isin(["Rotavirus", "Campylobacter jejuni", "Cryptosporidium parvum"])]
    # Querying dose response parameters based on pathogen inflow
    dr_models = read_frame(DoseResponse.objects.filter(pathogen__in=Pathogen.objects.filter(pathogen__in=df_inflow["pathogen__pathogen"])))

    # Querying for Logremoval based on selected treatments
    df_treatment=read_frame(LogRemoval.objects.filter(treatment__in=ra.treatment.all()).values("min", "max", "treatment__name", "pathogen_group__pathogen_group"))
    
    # Summarizing treatment to treatment max and treatment min
    # df_treatment_summary=df_treatment.groupby(["pathogen_group__pathogen_group"]).sum().reset_index()
    df_treatment_summary=df_treatment.groupby(["pathogen_group__pathogen_group"])[["min", "max"]].apply(lambda x : x.sum()).reset_index()
    results = pd.DataFrame()

    for index, row in df_inflow.iterrows():
        d = df_inflow.loc[df_inflow["pathogen__pathogen"] ==row["pathogen__pathogen"]]
        dr = dr_models.loc[dr_models["pathogen"]==row["pathogen__pathogen"]]

        if row["pathogen__pathogen"] == "Rotavirus":
            selector = "Viruses" 
        elif row["pathogen__pathogen"]== "Cryptosporidium parvum":
            selector = "Protozoa"
        else:
            selector = "Bacteria"
        #result.append(selector)

        df_treat = df_treatment_summary[df_treatment_summary["pathogen_group__pathogen_group"]==selector]



        risk_df = pd.DataFrame({"inflow": np.random.normal(loc=(np.log10(float(d["min"])+10**(-8))+np.log10(float(d["max"]) ))/2, 
                                                        scale = (np.log10(float(d["max"]))-np.log10(float(d["min"])+10**(-8) ))/4,  
                                                        size = 10000),
                            "LRV": np.random.uniform(low= df_treat["min"], 
                                                     high= df_treat["min"], 
                                                     size= 10000),
                            "LRVmax": np.random.uniform(low= df_treat["max"], 
                                                     high= df_treat["max"], 
                                                     size= 10000)})
        risk_df["outflow"]=risk_df["inflow"] - risk_df["LRV"]
        risk_df["outflow_min"]=risk_df["inflow"] - risk_df["LRVmax"]

        risk_df["dose"] = (10**risk_df["outflow"])*float(ra.exposure.volume_per_event)
        risk_df["dose_min"] = (10**risk_df["outflow_min"])*float(ra.exposure.volume_per_event)

        if selector != "Protozoa":
            risk_df["probs"] = 1 - (1 + (risk_df["dose"]) * (2 ** (1/float(dr.alpha)) - 1)/float(dr.n50)) ** -float(dr.alpha)
            risk_df["probs_min"] = 1 - (1 + (risk_df["dose_min"]) * (2 ** (1/float(dr.alpha)) - 1)/float(dr.n50)) ** -float(dr.alpha)

        else:
            risk_df["probs"] = 1 - np.exp(-float(dr.k)*(risk_df["dose"]))
            risk_df["probs_min"] = 1 - np.exp(-float(dr.k)*(risk_df["dose_min"]))

        results[row["pathogen__pathogen"]+"_MinimumLRV"] = [annual_risk(int(ra.exposure.events_per_year), risk_df["probs"] ) for _ in range(1000)]
        results[row["pathogen__pathogen"]+"_MaximumLRV"] = [annual_risk(int(ra.exposure.events_per_year), risk_df["probs_min"] ) for _ in range(1000)]


    results_long = pd.melt(results)
    results_long["log probability"] = np.log10(results_long["value"])
    return results_long

In [111]:
ras = RiskAssessment.objects.all()[:5]

In [112]:
results = []
for i in range(len(ras)):
    sim = simulate_risk(ras[i])
    sim["Assessment"] = ras[i].name
    results.append(sim)
  
df = pd.concat(results)

In [19]:
ra = RiskAssessment.objects.get(name = "Braunschweig status quo")

In [21]:
sim = simulate_risk(ra)

KeyError: 'min'

In [43]:
df_inflow = read_frame(Inflow.objects.filter(water_source=ra.source).values("min", "max", "pathogen__pathogen", "water_source__water_source_name"))
df_inflow = df_inflow[df_inflow["pathogen__pathogen"].isin(["Rotavirus", "Campylobacter jejuni", "Cryptosporidium parvum"])]


In [44]:
df_inflow

,min,max,pathogen__pathogen,water_source__water_source_name
4,50.00000000,5000.00000000,Rotavirus,"sewage, raw"
5,100.00000000,1000000.00000000,Campylobacter jejuni,"sewage, raw"
6,1.00000000,10000.00000000,Cryptosporidium parvum,"sewage, raw"


In [46]:
    # Querying dose response parameters based on pathogen inflow
dr_models = read_frame(DoseResponse.objects.filter(pathogen__in=Pathogen.objects.filter(pathogen__in=df_inflow["pathogen__pathogen"])))

    # Querying for Logremoval based on selected treatments
df_treatment=read_frame(LogRemoval.objects.filter(treatment__in=ra.treatment.all()).values("min", "max", "treatment__name", "pathogen_group__pathogen_group"))
 

In [48]:
df_treatment

,min,max,treatment__name,pathogen_group__pathogen_group
0,0.0,0.5,Primary treatment,Bacteria
1,0.0,0.1,Primary treatment,Viruses
2,0.0,1.0,Primary treatment,Protozoa
3,1.0,3.0,Secondary treatment,Bacteria
4,0.5,2.0,Secondary treatment,Viruses
5,0.5,1.5,Secondary treatment,Protozoa


In [59]:
df_treatment

,min,max,treatment__name,pathogen_group__pathogen_group
0,0.0,0.5,Primary treatment,Bacteria
1,0.0,0.1,Primary treatment,Viruses
2,0.0,1.0,Primary treatment,Protozoa
3,1.0,3.0,Secondary treatment,Bacteria
4,0.5,2.0,Secondary treatment,Viruses
5,0.5,1.5,Secondary treatment,Protozoa


In [69]:
df_treatment.groupby(["pathogen_group__pathogen_group"]).sum()

""
pathogen_group__pathogen_group
Bacteria
Protozoa
Viruses


In [79]:
df_treatment_summary=df_treatment.groupby("pathogen_group__pathogen_group").sum().reset_index()


In [88]:
df_treatment.groupby(["pathogen_group__pathogen_group"]).sum()

""
pathogen_group__pathogen_group
Bacteria
Protozoa
Viruses


In [103]:
df_treatment.groupby("treatment__name")["max"].sum().reset_index()

,treatment__name,max
0,Primary treatment,1.6
1,Secondary treatment,6.5


In [109]:
df_treatment.groupby(["pathogen_group__pathogen_group"])[["min", "max"]].apply(lambda x : x.sum()).reset_index()

,pathogen_group__pathogen_group,min,max
0,Bacteria,1.0,3.5
1,Protozoa,0.5,2.5
2,Viruses,0.5,2.1


In [86]:

results = pd.DataFrame()

In [113]:
df_treatment.groupby(["pathogen_group__pathogen_group"])[["min", "max"]].sum().reset_index()

,pathogen_group__pathogen_group
0,Bacteria
1,Protozoa
2,Viruses


In [73]:
d = df_inflow.loc[df_inflow["pathogen__pathogen"] =="Rotavirus"]

In [75]:
df_treatment

,min,max,treatment__name,pathogen_group__pathogen_group
0,0.0,0.5,Primary treatment,Bacteria
1,0.0,0.1,Primary treatment,Viruses
2,0.0,1.0,Primary treatment,Protozoa
3,1.0,3.0,Secondary treatment,Bacteria
4,0.5,2.0,Secondary treatment,Viruses
5,0.5,1.5,Secondary treatment,Protozoa


In [63]:
dr = dr_models.loc[dr_models["pathogen"]=="Rotavirus"]

In [64]:
dr

,id,pathogen,bestfitmodel,k,alpha,n50,hosttype,doseunits,route,response,reference
1,32,Rotavirus,beta-Poisson,0E-10,0.2530000000,6.1700000000,human,FFU,oral,infection,"Ward et al, 1986"


In [65]:
selector = "Viruses"

In [66]:
df_treat = df_treatment_summary[df_treatment_summary["pathogen_group__pathogen_group"]==selector]

KeyError: 'pathogen_group__pathogen_group'

In [67]:
df_treatment_summary

""
pathogen_group__pathogen_group
Bacteria
Protozoa
Viruses


In [33]:
risk_df = pd.DataFrame({"inflow": np.random.normal(loc=(np.log10(float(d["min"])+10**(-8))+np.log10(float(d["max"]) ))/2, 
                                                        scale = (np.log10(float(d["max"]))-np.log10(float(d["min"])+10**(-8) ))/4,  
                                                        size = 10000),
                            "LRV": np.random.uniform(low= df_treat["min"], 
                                                     high= df_treat["min"], 
                                                     size= 10000),
                            "LRVmax": np.random.uniform(low= df_treat["max"], 
                                                     high= df_treat["max"], 
                                                     size= 10000)})

KeyError: 'min'

In [42]:
df_treat

,pathogen_group__pathogen_group
2,Viruses


In [ ]:
    # Querying dose response parameters based on pathogen inflow
    dr_models = read_frame(DoseResponse.objects.filter(pathogen__in=Pathogen.objects.filter(pathogen__in=df_inflow["pathogen__pathogen"])))

    # Querying for Logremoval based on selected treatments
    df_treatment=read_frame(LogRemoval.objects.filter(treatment__in=ra.treatment.all()).values("min", "max", "treatment__name", "pathogen_group__pathogen_group"))
    
    # Summarizing treatment to treatment max and treatment min
    df_treatment_summary=df_treatment.groupby(["pathogen_group__pathogen_group"]).sum().reset_index()

    results = pd.DataFrame()

    for index, row in df_inflow.iterrows():
        d = df_inflow.loc[df_inflow["pathogen__pathogen"] ==row["pathogen__pathogen"]]
        dr = dr_models.loc[dr_models["pathogen"]==row["pathogen__pathogen"]]

        if row["pathogen__pathogen"] == "Rotavirus":
            selector = "Viruses" 
        elif row["pathogen__pathogen"]== "Cryptosporidium parvum":
            selector = "Protozoa"
        else:
            selector = "Bacteria"
        #result.append(selector)

        df_treat = df_treatment_summary[df_treatment_summary["pathogen_group__pathogen_group"]==selector]



        risk_df = pd.DataFrame({"inflow": np.random.normal(loc=(np.log10(float(d["min"])+10**(-8))+np.log10(float(d["max"]) ))/2, 
                                                        scale = (np.log10(float(d["max"]))-np.log10(float(d["min"])+10**(-8) ))/4,  
                                                        size = 10000),
                            "LRV": np.random.uniform(low= df_treat["min"], 
                                                     high= df_treat["min"], 
                                                     size= 10000),
                            "LRVmax": np.random.uniform(low= df_treat["max"], 
                                                     high= df_treat["max"], 
                                                     size= 10000)})
        risk_df["outflow"]=risk_df["inflow"] - risk_df["LRV"]
        risk_df["outflow_min"]=risk_df["inflow"] - risk_df["LRVmax"]

        risk_df["dose"] = (10**risk_df["outflow"])*float(ra.exposure.volume_per_event)
        risk_df["dose_min"] = (10**risk_df["outflow_min"])*float(ra.exposure.volume_per_event)

        if selector != "Protozoa":
            risk_df["probs"] = 1 - (1 + (risk_df["dose"]) * (2 ** (1/float(dr.alpha)) - 1)/float(dr.n50)) ** -float(dr.alpha)
            risk_df["probs_min"] = 1 - (1 + (risk_df["dose_min"]) * (2 ** (1/float(dr.alpha)) - 1)/float(dr.n50)) ** -float(dr.alpha)

        else:
            risk_df["probs"] = 1 - np.exp(-float(dr.k)*(risk_df["dose"]))
            risk_df["probs_min"] = 1 - np.exp(-float(dr.k)*(risk_df["dose_min"]))

        results[row["pathogen__pathogen"]+"_MinimumLRV"] = [annual_risk(int(ra.exposure.events_per_year), risk_df["probs"] ) for _ in range(1000)]
        results[row["pathogen__pathogen"]+"_MaximumLRV"] = [annual_risk(int(ra.exposure.events_per_year), risk_df["probs_min"] ) for _ in range(1000)]


In [13]:
fig = px.box(df, x="variable", y="value", color="Assessment", 
                            points="all",  log_y =True, 
                            title="Risk as probability of infection per year",
                            color_discrete_sequence=["#004254", "#007c9e", "#a3d1ec","#3494ae","#00B8eb"])


fig.update_layout(
    font_family="Helvetica Neue, Helvetica, Arial, sans-serif",
    font_color="black",
    title = {'text':'Risk assessment as probability of infection per year'},
    xaxis_title = "Reference Pathogen",
    yaxis_title = "Probability of infection per year",
    #markersize= 12,
    )

fig.update_layout(legend=dict(
                 orientation="h",
                 yanchor="top",
                 y=-.1,
                 xanchor="left",
                x=0))


fig.update_traces(marker_size = 8)#['#75c3ff', "red"],#, marker_line_color='#212c52',


fig.show()



In [17]:
df["pathogen"] = df["variable"].str.split("_", expand=True)[0]
df["stat"] = df["variable"].str.split("_", expand=True)[1]

In [36]:
dfmin = df.groupby(["pathogen", "Assessment"]).min("value").reset_index().assign(stat = "min")
dfmax = df.groupby(["pathogen", "Assessment"]).max("value").reset_index().assign(stat = "max")


In [112]:
df_summary = dfmin.append(dfmax).sort_values(by = "value", ascending = False)

In [114]:
fig = px.box(df_summary, x="Assessment", y="value", 
                            color="pathogen", 
                          
                            log_y =True, 
                      labels={
                             "pathogen": "Reference pathogen"
                             },
                         
                            title="Risk as probability of infection per year",
                            color_discrete_sequence=["#004254", "#007c9e", "#a3d1ec","#3494ae","#00B8eb"])



fig.update_layout(
    font_family="Helvetica Neue, Helvetica, Arial, sans-serif",
    font_color="black",
    title = {'text':'Risk assessment as probability of infection per year'},
    xaxis_title = "",
    yaxis_title = "Probability of infection per year",
    #markersize= 12,
    )


fig.update_layout(legend=dict(
                 orientation="h",
                 yanchor="top",
                 #text= "Reference pathogen",
                 y=-.1,
                 xanchor="left",
                x=0))

fig.add_hline(y=0.0001, line_dash="dashdot", line=dict(color="LightSeaGreen", width = 3),
              #annotation_text="Jan 1, 2018 baseline", 
              #annotation_position="bottom right",
              #annotation_font_size=20,
              #annotation_font_color="blue"
             )

fig.update_traces(marker_size = 8,hovertemplate=None, hoverinfo="skip",
                 line=dict(width=0))#['#75c3ff', "red"],#, marker_line_color='#212c52',
#fig.update_xaxes(type='category')
#fig.update_xaxes(range = [-10, 5])
fig.show()

In [117]:
r = RiskAssessment.objects.get(name = "New risk assessment")

In [118]:
simulate_risk(r)

,variable,value,log probability
0,Rotavirus_MinimumLRV,2.121407e-03,-2.673376
1,Rotavirus_MinimumLRV,6.044444e-03,-2.218644
2,Rotavirus_MinimumLRV,4.103047e-03,-2.386893
3,Rotavirus_MinimumLRV,2.361226e-03,-2.626862
4,Rotavirus_MinimumLRV,1.915020e-03,-2.717827
...,...,...,...
5995,Cryptosporidium parvum_MaximumLRV,1.250481e-10,-9.902923
5996,Cryptosporidium parvum_MaximumLRV,3.200052e-10,-9.494843
5997,Cryptosporidium parvum_MaximumLRV,4.896653e-10,-9.310101
5998,Cryptosporidium parvum_MaximumLRV,2.695102e-09,-8.569425
